In [0]:
#https://keras.io/
!pip install -q keras
import keras

In [3]:
#모든 장치목록을 확인 GPU, CPU
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15199183654642704024
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9985002106750786283
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2917939871425418666
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5338071957948089527
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [0]:
from google.colab import drive

drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


In [0]:
#resnet152, optimizer.lr = 0.0001로 변경한 코드
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
import matplotlib.pyplot as plt
import numpy as np
import math
 
 

# 적은 이미지에서 최대한 많은 정보를 뽑아내서 학습할 수 있도록 데이터들을 전처리
# 학습 도중에 이미지에 임의 변형 및 정규화 적용
# 실수 표현
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
 
# 이미지가 들어있는 dir을 지정
# os.path.join 경로를 병합하여 새로운 경로 생성
train_dir = os.path.join('/content/gdrive/My Drive/resnet1/imagegenerator2/train')
test_dir = os.path.join('/content/gdrive/My Drive/resnet1/test')

# 정규화할 데이터가 있는 dir과 각종 속성 지정
# batch_size: sample data중 한번에 넘겨주는 데이터의 수
# 이미지 파일을 실제 사용할 수 있는 값으로 바꿈
train_generator = train_datagen.flow_from_directory(
    train_dir, batch_size=4, target_size=(240, 320), color_mode='rgb', class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(
    test_dir, batch_size=2, target_size=(240, 320), color_mode='rgb', class_mode = 'categorical')
 
# number of classes
K = 2
 
# input 지정
input_tensor = Input(shape=(240, 320, 3), dtype='float32', name='input')
 
# zeropadding: 이미지주위를 0으로 둘러싸 이미지 크기가 줄어들지 않도록 한다.
# Conv2D: 컨볼루션 연산을 구현 64:컨볼루션 필터의 수, (7,7): 필터의 행과 열의 크기
# Normalization: 데이터의 범위를 사용자가 원하는 범위로 제한하는 것  ex)pixel 0~255를 0~1.0값으로
# 학습을 더 빨리하고 Local optimum에 빠지는 가능성을 줄이기 위해서
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   
 
    
#Maxpooling: (3,3) 가로세로 크기 3X3중 가장 큰 값, strides는 2
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x
 
  
def conv3_layer(x):        
    shortcut = x    
    # resnet152 변경부분: 50은 4번 반복, 8번 반복으로 증가
    for i in range(8):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x
 
 
 
def conv4_layer(x):
    shortcut = x        
    # resnet152 변경부분: 50은 6번 반복, 36번 반복으로 증가
    for i in range(36):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x
 
 
 
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x
 
 
 
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

#공간적 데이터에 대한 글로벌 최대값 풀링 작업 
x = GlobalAveragePooling2D()(x)
#Dense: 입력(출력 이전 층)과 출력을 모두 연결, K는 출력 뉴런의 수를 의미
output_tensor = Dense(K, activation='softmax')(x)
 
#생성한 인풋, 각종 레이어, 아웃풋으로 모두 연결하여 하나의 모델로 정의
#아래를 실행하겠다.
resnet152 = Model(input_tensor, output_tensor)
#최적화함수: adam, 오류(손실)함수:crossentropy
resnet152.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#기존의 learningrate 값은 0.00005, 0.0001로 변경시킴.
#학습률
resnet152.optimizer.lr = 0.0001
resnet152.fit(
    train_generator, 
    steps_per_epoch = 25,
    epochs = 50,
    validation_data = test_generator,
    #validation 모델을 평가하기 위한 데이터
    validation_steps = 20)

scores = resnet152.evaluate_generator(test_generator, steps = 20)
print("\n 테스트 데이터 %s : %.2f%% \n\n" % (resnet152.metrics_names[1], scores[1]*100))

out = resnet152.predict_generator(test_generator, steps = 20)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(out)

Found 200 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/50
25/25 [==============================] - 55s 2s/step - loss: 0.7721 - acc: 0.6700 - val_loss: 2.5136 - val_acc: 0.5000
Epoch 2/50
25/25 [==============================] - 6s 245ms/step - loss: 0.6014 - acc: 0.7400 - val_loss: 1.1039 - val_acc: 0.8250
Epoch 3/50
25/25 [==============================] - 6s 246ms/step - loss: 0.3998 - acc: 0.8600 - val_loss: 1.7443 - val_acc: 0.7500
Epoch 4/50
25/25 [==============================] - 6s 243ms/step - loss: 0.4620 - acc: 0.8200 - val_loss: 2.6405 - val_acc: 0.5750
Epoch 5/50
25/25 [==============================] - 6s 246ms/step - loss: 0.5350 - acc: 0.7900 - val_loss: 0.6538 - val_acc: 0.7250
Epoch 6/50
25/25 [==============================] - 6s 246ms/step - loss: 0.2740 - acc: 0.8900 - val_loss: 1.9223 - val_acc: 0.6000
Epoch 7/50
25/25 [==============================] - 6s 247ms/step - loss: 0.6230 - acc: 0.7500 - val_loss: 1.5179 - val_acc: 0.72